In [364]:
import pandas as pd
import numpy as np
import operator

##import dataframe
df = pd.DataFrame()
df = pd.read_csv('dssp.csv', index_col=0, skiprows=2)

schedule_width = len(df.columns)

## create minimum number of people on each team array
min_members = pd.DataFrame()
min_members = pd.read_csv('dssp.csv', index_col=0, nrows=1)

## from https://stackoverflow.com/questions/44722436/create-pandas-dataframes-from-unique-values-in-one-column
## create sport matrix
sports = df['Sport'].unique().tolist()
sports_dict = {sport: df.loc[df['Sport'] == sport] for sport in sports}
#print sports_dict

## create league dataframe
for sport in sports_dict:
    leagues = sports_dict[sport]['League'].unique().tolist()
    sports_dict[sport] = {league: df.loc[(df['League'] == league) & (df['Sport'] == sport)] for league in leagues}
#print sports_dict

## create teams dataframe
for sport in sports_dict:
    for league in sports_dict[sport]:
        teams = sports_dict[sport][league]['Team'].unique().tolist()
        sports_dict[sport][league] = {team: df.loc[(df['Team'] == team) & (df['League'] == league) & (df['Sport'] == sport)] for team in teams}
#print sports_dict 
        
## create games (intersection of teams) dataframe
#int_teams = {("Soccer", 1, 1,2),("Soccer", 1, 3,4),("Soccer", 1, 1,3),("Soccer", 1, 1,4),("Soccer", 1, 1,4),("Soccer", 1, 2,3)}
int_teams = [(1,1,2),(1,3,4),(1,5,6),(1,7,8),(2,1,4),(2,2,6),(2,3,8),(2,5,7),(3,1,6),(3,4,8),(3,2,7),(3,3,5),(4,1,8),(4,6,7),(4,4,5),(4,2,3),(5,1,7),(5,8,5),(5,6,3),(5,4,2),(6,1,5),(6,7,3),(6,8,2),(6,6,4),(7,1,3),(7,5,2),(7,7,4),(7,8,6)]

## find the least available intersections
## (lowest availability is the minimum sum of a teams availability)
avail_arr = {}
for sport in sports_dict:
    avail_arr[sport]={}
    for league in sports_dict[sport]:
        avail_arr[sport][league]=[]
        for each in int_teams:
            #print each
            #if (each[0] == week):
                try:
                    #from https://stackoverflow.com/questions/25748683/pandas-sum-dataframe-rows-for-given-columns
                    #print sports_dict[sport][i].iloc[:, 4:schedule_width].values.sum()
                    if ((sports_dict[sport][league][each[1]].iloc[:, 4:schedule_width].values.sum() > 0) & (sports_dict[sport][league][each[1]].iloc[:, 4:schedule_width].values.sum() > 0)):
                        avail_arr[sport][league].append([each[0], each[1], each[2], sports_dict[sport][league][each[1]].iloc[:, 4:schedule_width].values.sum() + sports_dict[sport][league][each[2]].iloc[:, 4:schedule_width].values.sum()])
                    #print sports_dict[sport][league][each[1]]
                    #print avail_arr[sport][league][each[2]]
                    print sport, league, "week", each[0], each
                    
                except:
                    pass
                    #avail_arr[sport].append(['Week ' + str(each[1]), each[2], each[3], "Missing Team"])
                #print avail_arr[sport]
#print avail_arr
#sort array by availability
#print avail_arr
for sport in avail_arr:
    for league in avail_arr[sport]:
        #print avail_arr[sport]
        # from https://stackoverflow.com/questions/10695139/sort-a-list-of-tuples-by-2nd-item-integer-value
        avail_arr[sport][league] = sorted(avail_arr[sport][league], key=lambda x: x[3])
print avail_arr

#Create games dictionary
#temp_games= [None] * 105
games_dict = {}
for i in xrange(12):
    games_dict[i] = []
print games_dict
#print sports_dict
print ""
print ""
# iterate through weeks
# Find best time for each team to play
# Make sure that the total games per period < 3
# make sure both teams have minimum amout of players
# else, go maximum 
#print df
week = 1
while (week < 8):
    for sport in avail_arr:
        for league in avail_arr[sport]:
            for each in avail_arr[sport][league]:
                #print "each", each
                #for time in sports_dict[sport][league][each[1]].iloc[:, 4:schedule_width].values
                #print time
                if (each [0] == week):
                    try:
                        # for both teams, loop through all times and find the sums of availability
                        # find max sum
                        # check if both are > min
                        # if yes, then print the game to corresponding location in actual_games dataframe for this week
                        # exit loop once a data has been entered

                        #print sports_dict[sport][league][each[1]].iloc[:, 4:schedule_width]
                        team1 = {}
                        for time in sports_dict[sport][league][each[1]].iloc[:, 4:schedule_width]:
                            #print time
                            #print sports_dict[sport][league][each[1]][time].values.sum()
                            team1[time]=(sports_dict[sport][league][each[1]][time].values.sum())
                        #print 'team1 array', team1
                        team2 = {}
                        #print sports_dict[sport][league][each[2]].iloc[:, 4:schedule_width]
                        for time in sports_dict[sport][league][each[2]].iloc[:, 4:schedule_width]:
                            #print time
                            #print sports_dict[sport][league][each[2]][time].values.sum()
                            team2[time]=(sports_dict[sport][league][each[2]][time].values.sum())
                        #print 'team2 array', team2
                        
                        for game_time in team1:
                            #print "team1", team1[game_time], game_time, sport, league, "week", week, each
                            #print "team2", team2[game_time], game_time, sport, league, "week", week, each
                            if ((team1[game_time] > 3) & (team2[game_time] > 3)):
                                try:
                                    if (len(games_dict[week]) < 3):
                                        games_dict[week].append((sport, league, each[1],each[2],game_time))
                                        #print "dict", games_dict[week]
                                        break
                                except:
                                    pass
                    except:
                        pass
    week = week + 1
#print games_dict

# create dataframe contain games played 
# (each cell will be an array of games info)
games_actual = {}
week = 1
while (week < 8):
    games_actual['Week' + str(week)] = pd.DataFrame(index=['9:00','10:00','11:00','12:00','13:00','14:00','15:00','16:00','17:00','18:00','19:00','20:00','21:00','22:00','23:00']
,columns=["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"])
    games_actual['Week' + str(week)].fillna(value={})
    week = week + 1
#print games_actual 

## Fill games_actual
for week in games_dict:
    for each in games_dict[week]:
        temp_string = each[4].split()
        print "temp", temp_string, "Week"+str(week)
        print games_actual["Week"+str(week)].loc[temp_string[1],temp_string[0]]
        games_actual["Week"+str(week)].loc[temp_string[1],temp_string[0]] = each
        
print ""
print "games actual"
print games_actual["Week1"] 
print ""
print ""
## From https://stackoverflow.com/questions/42370977/how-to-save-a-new-sheet-in-an-existing-excel-file-using-pandas
## print each game array into a excel
## each tab of excel is a week with all relevant games
writer = pd.ExcelWriter("output.xlsx", engine = 'xlsxwriter')
week = 1
for each in games_actual:
    games_actual['Week' + str(week)].to_excel(writer, sheet_name = 'Week' + str(week))
    week = week + 1
writer.save()
writer.close()


Soccer Beginner week 1 (1, 1, 2)
Hockey Beginner week 1 (1, 1, 2)
Hockey Beginner week 4 (4, 2, 3)
Hockey Beginner week 7 (7, 1, 3)
{'Soccer': {'Beginner': [[1, 1, 2, 30]]}, 'Hockey': {'Beginner': [[1, 1, 2, 6], [7, 1, 3, 18], [4, 2, 3, 20]]}}
{0: [], 1: [], 2: [], 3: [], 4: [], 5: [], 6: [], 7: [], 8: [], 9: [], 10: [], 11: []}


temp ['Monday', '18:00'] Week1
nan
temp ['Monday', '18:00'] Week4
nan

games actual
                                       Monday Tuesday Wednesday Thursday  \
9:00                                      NaN     NaN       NaN      NaN   
10:00                                     NaN     NaN       NaN      NaN   
11:00                                     NaN     NaN       NaN      NaN   
12:00                                     NaN     NaN       NaN      NaN   
13:00                                     NaN     NaN       NaN      NaN   
14:00                                     NaN     NaN       NaN      NaN   
15:00                                     NaN     N